# Brianne Bell
## ADS 509 Module 4: Naive Bayes on Political Text
### February 6, 2023

In this notebook we use Naive Bayes to explore and classify political data. See the `README.md` for full details.

In [1]:
import sqlite3
import nltk
import random
import numpy as np
from collections import Counter, defaultdict

import os
import re
from nltk.corpus import stopwords
from string import punctuation

# Feel free to include your text patterns functions
#from text_functions_solutions import clean_tokenize, get_patterns

In [2]:
# Text patterns functions from module 3:
# Some punctuation variations
punctuation = set(punctuation) # speeds up comparison
tw_punct = punctuation - {"#"}

# Stopwords
sw = stopwords.words("english")

# Two useful regex
whitespace_pattern = re.compile(r"\s+")

In [3]:
convention_db = sqlite3.connect("2020_Conventions.db")
convention_cur = convention_db.cursor()

In [4]:
# getting table name(s):
con_tables = convention_db.execute("SELECT name FROM sqlite_master WHERE type='table';")
for name in con_tables.fetchall():
    print(name[0])
# https://stackoverflow.com/questions/34570260/how-to-get-table-names-using-sqlite3-through-python

conventions


In [5]:
# seeing what is in the <conventions> table
convention_db.row_factory = sqlite3.Row
concursor = convention_db.execute("SELECT * FROM conventions") # select all from table
conrow = concursor.fetchone()
connames = conrow.keys()
print(connames)

# https://stackoverflow.com/questions/7831371/is-there-a-way-to-get-a-list-of-column-names-in-sqlite

['party', 'night', 'speaker', 'speaker_count', 'time', 'text', 'text_len', 'file']


The convention database is in table: conventions. This table has 'party', 'night', 'speaker', 'speaker_count', 'time', 'text', 'text_len', and 'file'. We will want 'party' and 'text' (as mentioned in next section).

### Part 1: Exploratory Naive Bayes

We'll first build a NB model on the convention data itself, as a way to understand what words distinguish between the two parties. This is analogous to what we did in the "Comparing Groups" class work. First, pull in the text 
for each party and prepare it for use in Naive Bayes.  

In [6]:
convention_data = []

# fill this list up with items that are themselves lists. The 
# first element in the sublist should be the cleaned and tokenized
# text in a single string. The second element should be the party. 

query_results = convention_cur.execute(
                            '''
                            SELECT text, party FROM conventions
                            ''')

# Functions for cleaning text:
## Removing Stop Tokens
def remove_stop(tokens) :
    # modify this function to remove stopwords
    return [i for i in tokens if i.lower() not in sw]

## Removing Punctiation (but not hashtags)
def remove_punctuation(text, punct_set=tw_punct) : 
    return("".join([ch for ch in text if ch not in punct_set]))

## Tokenization Function with split on whitespace
def tokenize(text) : 
    """ Splitting on whitespace rather than the book's tokenize function. That 
        function will drop tokens like '#hashtag' or '2A', which we need for Twitter. """
    
    # modify this function to return tokens
    return(i for i in whitespace_pattern.split(text))

## Prepare Function
def prepare(text, pipeline) : 
    tokens = str(text)
    
    for transform in pipeline : 
        tokens = transform(tokens)
        
    return(tokens)

my_pipeline = [str.lower, remove_punctuation, tokenize, remove_stop]

# moving on to calling functions
for row in query_results :
    text, party = row # each row of query result will have text and party
    prep_text = prepare(text, pipeline= my_pipeline) # calling pipeline of functions for text
    
    # making tokens into string of words:
    speeches = " ".join(prep_text)
    convention_data.append([speeches, party])
    

Let's look at some random entries and see if they look right. 

In [7]:
random.choices(convention_data,k=10)

[['three bond forged sorrow expanded joy jill entered', 'Democratic'],
 ['thanks voters across country red states blue states – n fried kimpson j nez r',
  'Democratic'],
 ['he’ll treat everyone respect matter', 'Democratic'],
 ['almost four years ago went election day completely underestimated despite said year know americans go polls behalf families economy national security children’s future vote ideas partisan vote common sense vote goals hopes believe believe need husband’s leadership ever order bring us back greatest economy strongest country ever known god bless families god bless united states america related transcripts iowa gov kim reynolds press conference transcript september 16 • 46 mins ago press secretary kayleigh mcenany white house press conference transcript september 16 • 58 mins ago doj press conference transcript september 16 charges 5 chinese nationals • 2 hours ago rnc night 2 rnc night 2 first lady melania trump secretary state mike pompeo eric trump speak secon

If that looks good, we now need to make our function to turn these into features. In my solution, I wanted to keep the number of features reasonable, so I only used words that occur at least `word_cutoff` times. Here's the code to test that if you want it. 

In [8]:
word_cutoff = 5

tokens = [w for t, p in convention_data for w in t.split()]

word_dist = nltk.FreqDist(tokens)

feature_words = set()

for word, count in word_dist.items() :
    if count > word_cutoff :
        feature_words.add(word)
        
print(f"With a word cutoff of {word_cutoff}, we have {len(feature_words)} as features in the model.")

With a word cutoff of 5, we have 2391 as features in the model.


In [9]:
def conv_features(text,fw) :
    """Given some text, this returns a dictionary holding the
       feature words.
       
       Args: 
            * text: a piece of text in a continuous string. Assumes
            text has been cleaned and case folded.
            * fw: the *feature words* that we're considering. A word 
            in `text` must be in fw in order to be returned. This 
            prevents us from considering very rarely occurring words.
        
       Returns: 
            A dictionary with the words in `text` that appear in `fw`. 
            Words are only counted once. 
            If `text` were "quick quick brown fox" and `fw` = {'quick','fox','jumps'},
            then this would return a dictionary of 
            {'quick' : True,
             'fox' :    True}
        
    """
    
    # Your code here
    
    ret_dict = dict()
    for i in text.split(): # looping through
        if i in fw: # checking to see if i is in fw (feature word)
            ret_dict[i] = True # if it is then respond with true
    
    return(ret_dict)

In [10]:
# assert question 1
assert(len(feature_words)>0)


In [11]:
# assert question 2
assert(conv_features("donald is the president",feature_words)==
       {'donald':True,'president':True})


In [12]:
# assert question 3
assert(conv_features("people are american in america",feature_words)==
                     {'america':True,'american':True,"people":True})

Now we'll build our feature set. Out of curiosity I did a train/test split to see how accurate the classifier was, but we don't strictly need to since this analysis is exploratory. 

In [13]:
featuresets = [(conv_features(text,feature_words), party) for (text, party) in convention_data]

In [14]:
random.seed(20220507)
random.shuffle(featuresets)

test_size = 500

In [15]:
test_set, train_set = featuresets[:test_size], featuresets[test_size:]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.5


In [16]:
classifier.show_most_informative_features(25)

Most Informative Features
                   china = True           Republ : Democr =     25.8 : 1.0
                   votes = True           Democr : Republ =     23.8 : 1.0
             enforcement = True           Republ : Democr =     21.5 : 1.0
                 destroy = True           Republ : Democr =     19.2 : 1.0
                freedoms = True           Republ : Democr =     18.2 : 1.0
                 climate = True           Democr : Republ =     17.8 : 1.0
                supports = True           Republ : Democr =     17.1 : 1.0
                   crime = True           Republ : Democr =     16.1 : 1.0
                   media = True           Republ : Democr =     14.9 : 1.0
                 beliefs = True           Republ : Democr =     13.0 : 1.0
               countries = True           Republ : Democr =     13.0 : 1.0
                 defense = True           Republ : Democr =     13.0 : 1.0
                    isis = True           Republ : Democr =     13.0 : 1.0

Write a little prose here about what you see in the classifier. Anything odd or interesting?

### My Observations

Republications dominate the top 25 informative features by using them 23 of the 25 times. Democrats featured the top feature when using 'votes' and 'climate' which reflects the Democratic party's concern about climate change. It is a bit strange that the top feature word is 'china' while the 25th feature word is 'amendment', one would hope that is would be the opposite. There are also a lot of negative (in my opinion) words like 'crime', 'destroyed', 'enemy' and 'isis' in this list. I am also not sure why 'abraham' is top feature. 

---

## Part 2: Classifying Congressional Tweets

In this part we apply the classifer we just built to a set of tweets by people running for congress
in 2018. These tweets are stored in the database `congressional_data.db`. That DB is funky, so I'll
give you the query I used to pull out the tweets. Note that this DB has some big tables and 
is unindexed, so the query takes a minute or two to run on my machine.

In [17]:
cong_db = sqlite3.connect("congressional_data.db")
cong_cur = cong_db.cursor()

In [18]:
results = cong_cur.execute(
        '''
           SELECT DISTINCT 
                  cd.candidate, 
                  cd.party,
                  tw.tweet_text
           FROM candidate_data cd 
           INNER JOIN tweets tw ON cd.twitter_handle = tw.handle 
               AND cd.candidate == tw.candidate 
               AND cd.district == tw.district
           WHERE cd.party in ('Republican','Democratic') 
               AND tw.tweet_text NOT LIKE '%RT%'
        ''')

results = list(results) # Just to store it, since the query is time consuming

In [27]:
tweet_data = []

# Now fill up tweet_data with sublists like we did on the convention speeches.
# moving on to calling functions
for row in results :
    candidate, party, tweet = row # each row of query result will have candidate, party, and tweet
    decode_tweet = tweet.decode('utf-8') # decoding the tweet text so it is usable
    prep_text = prepare(text= decode_tweet, pipeline= my_pipeline) # calling pipeline of functions for text
    
    # making tokens into string of words:
    ready_tweets = " ".join(prep_text)
    tweet_data.append([ready_tweets, party]) # classifier didn't like having candidate in there with it so removed


# Note that this may take a bit of time, since we have a lot of tweets.


There are a lot of tweets here. Let's take a random sample and see how our classifer does. I'm guessing it won't be too great given the performance on the convention speeches...

In [28]:
random.seed(20201014)

tweet_data_sample = random.choices(tweet_data,k=10)
tweet_data_sample

[['earlier today spoke house floor abt protecting health care women praised ppmarmonte work central coast httpstcowqgtrzt7vv',
  'Democratic'],
 ['go tribe #rallytogether httpstco0nxutfl9l5', 'Democratic'],
 ['apparently trump thinks easy students overwhelmed crushing burden debt pay student loans #trumpbudget httpstcockyqo5t0qh',
  'Democratic'],
 ['we’re grateful first responders rescue personnel firefighters police volunteers working tirelessly keep people safe provide muchneeded help putting lives line httpstcoezpv0vmiz3',
  'Republican'],
 ['let’s make even greater #kag 🇺🇸 httpstcoy9qozd5l2z', 'Republican'],
 ['1hr cavs tie series 22 im #allin216 repbarbaralee scared #roadtovictory',
  'Democratic'],
 ['congrats belliottsd new gig sd city hall glad continue serve… httpstcofkvmw3cqdi',
  'Democratic'],
 ['really close 3500 raised toward match right whoot that’s 7000 nonmath majors room 😂 help us get httpstcotu34c472sd httpstcoqsdqkypsmc',
  'Democratic'],
 ['today comment period po

In [29]:

for tweet, party in tweet_data_sample :
    #estimated_party = 'Gotta fill this in'
    # call in the conv_features function on tweet
    estimate_setup = conv_features(tweet, feature_words)
    # running setup through classifier from nltk:
    estimated_party = classifier.classify(estimate_setup)
    # Fill in the right-hand side above with code that estimates the actual party
    
    print(f"Here's our (cleaned) tweet: {tweet}")
    print(f"Actual party is {party} and our classifer says {estimated_party}.")
    print("")
    

Here's our (cleaned) tweet: earlier today spoke house floor abt protecting health care women praised ppmarmonte work central coast httpstcowqgtrzt7vv
Actual party is Democratic and our classifer says Republican.

Here's our (cleaned) tweet: go tribe #rallytogether httpstco0nxutfl9l5
Actual party is Democratic and our classifer says Democratic.

Here's our (cleaned) tweet: apparently trump thinks easy students overwhelmed crushing burden debt pay student loans #trumpbudget httpstcockyqo5t0qh
Actual party is Democratic and our classifer says Republican.

Here's our (cleaned) tweet: we’re grateful first responders rescue personnel firefighters police volunteers working tirelessly keep people safe provide muchneeded help putting lives line httpstcoezpv0vmiz3
Actual party is Republican and our classifer says Republican.

Here's our (cleaned) tweet: let’s make even greater #kag 🇺🇸 httpstcoy9qozd5l2z
Actual party is Republican and our classifer says Republican.

Here's our (cleaned) tweet: 1h

Now that we've looked at it some, let's score a bunch and see how we're doing.

In [30]:
# dictionary of counts by actual party and estimated party. 
# first key is actual, second is estimated
parties = ['Republican','Democratic']
results = defaultdict(lambda: defaultdict(int))

for p in parties :
    for p1 in parties :
        results[p][p1] = 0


num_to_score = 10000
random.shuffle(tweet_data)

for idx, tp in enumerate(tweet_data) :
    tweet, party = tp    
    # Now do the same thing as above, but we store the results rather
    # than printing them. 
   
    # get the estimated party
    #estimated_party = "Gotta fill this in"
    estimate_setup = conv_features(tweet, feature_words)
    estimated_party = classifier.classify(estimate_setup)
    
    results[party][estimated_party] += 1
    
    if idx > num_to_score : 
        break

In [31]:
results

defaultdict(<function __main__.<lambda>()>,
            {'Republican': defaultdict(int,
                         {'Republican': 3695, 'Democratic': 583}),
             'Democratic': defaultdict(int,
                         {'Republican': 4817, 'Democratic': 907})})

### Reflections

Similar to the speeches, most key words come from Republican speeches and most of the tweets are being classified as coming from Republicans since we used the same key words for classification. Looking back at the accuracy of the model (seen in this line of code: classifier = nltk.NaiveBayesClassifier.train(train_set)) the classifier was only 50% accurate. Once could argue you could flip a coin or classify all the tweeters as Republican and be right half the time, like with the model.  

---